<a href="https://colab.research.google.com/github/adc257/info4940-sitcom/blob/main/may13_human_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate


In [ ]:
!pip install openai

In [1]:
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import random
import json
import os

In [2]:
!git clone https://github.com/adc257/info4940-sitcom.git


from google.colab import drive
drive.mount('/content/drive')

fatal: destination path 'info4940-sitcom' already exists and is not an empty directory.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Defining some helper functions

In [3]:
def list_files(start_path):
    file_paths = []
    for root, dirs, files in os.walk(start_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    file_paths.sort()
    return file_paths

def fPathList_TO_DialogueWithContext(filePathList, nSentenceContext):
  humorous_inputs = []
  non_humorous_inputs = []

  # formatted_inputs = []
  # labels = []

  for episodePath in filePathList:

    with open(episodePath, 'r') as file:
      data = json.load(file)
      data = [info for (_, info) in data.items()]


      i = nSentenceContext
      while i < len(data):
        context = [data[j] for j in range(i-nSentenceContext, i)]
        target = data[i]

        if 'isHumor' in target:
          humorous_inputs.append(transformInput(context, target))
        else:
          non_humorous_inputs.append(transformInput(context, target))

        i += 1

  return humorous_inputs, non_humorous_inputs

def sampleEven(humorousData, nonHumorousData, sample_size_per_class, rand_seed):
  sample_size_per_class = min(sample_size_per_class, len(humorousData), len(nonHumorousData))

  sample_inputs = []
  sample_labels = []

  data = humorousData
  test_size = (len(data) - sample_size_per_class) / len(data)
  sample, _ = train_test_split(data, test_size=test_size, random_state=rand_seed)
  sample_inputs += sample
  sample_labels += [1]*len(sample)


  data = nonHumorousData
  test_size = (len(data) - sample_size_per_class) / len(data)
  sample, _ = train_test_split(data, test_size=test_size, random_state=rand_seed)
  sample_inputs += sample
  sample_labels += [0]*len(sample)

  return sample_inputs, sample_labels



In [4]:
def testGPT(inputs, labels, printResultsEveryN=50):
  resultsData = {
      "truePos": [],
      "trueNeg": [],
      "falsePos": [],
      "falseNeg": [],
      "other": [],
  }

  results = {
      "truePos": 0,
      "trueNeg": 0,
      "falsePos": 0,
      "falseNeg": 0,
      "other": 0,
  }

  correct = 0
  total = 0
  pred = []



  # terminators = [
  #     tokenizer.eos_token_id,
  #     tokenizer.convert_tokens_to_ids("<|eot_id|>")
  # ]

  for i in range(len(inputs)):
    input_ = inputs[i]
    label = labels[i]

    # input_ids = tokenizer.apply_chat_template(
    #   input,
    #   add_generation_prompt=True,
    #   return_tensors="pt"
    # ).to(model.device)

    # outputs = model.generate(
    #   input_ids,
    #   max_new_tokens=50,
    #   eos_token_id=terminators,
    #   do_sample=True,
    #   temperature=0.6,
    #   top_p=0.9,
    #   num_return_sequences=1,
    # )

    # response = outputs[0][input_ids.shape[-1]:]
    # response = tokenizer.decode(response, skip_special_tokens=True)

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=input_
    )

    response = completion.choices[0].message.content


    if str(response) == str(label):
      # true
      if str(label) == "1":
        # pos
        resultsData["truePos"].append(input_)
        results["truePos"] += 1
      elif str(label) == "0":
        # neg
        resultsData["trueNeg"].append(input_)
        results["trueNeg"] += 1
      else:
        # other
        resultsData["other"].append(input_)
        results["other"] += 1
      correct += 1
    else:
      # false
      if str(label) == "1":
        # pos, pred neg
        resultsData["falseNeg"].append(input_)
        results["falseNeg"] += 1
      elif str(label) == "0":
        # neg, pred pos
        resultsData["falsePos"].append(input_)
        results["falsePos"] += 1
      else:
        # other
        resultsData["other"].append(input_)
        results["other"] += 1
    pred.append(response)
    total += 1

    if i % printResultsEveryN == 0:
      print(f"Test: {i+1} of  {len(inputs)}")
      print(f"Success rate: {correct / total:.2%}\n")
      print("current results:", results)


  print(f"Final success rate: {correct / total:.2%}")
  print("final results:", results)

  # true = [str(i) for i in labels]
  report = None # classification_report(y_pred=pred, y_true=true, target_names=["Non-Humorous","Humorous"])

  return results, resultsData, pred, report

## Defining input formatter

In [5]:
def transformInput(context, target):
  messages = []

  separator = "\n"
  quotation_token = '"'

  # formatted_input = ""

  if (len(context) > 0):
    # if context...
    systemPrmpt = {
        'role': 'system',
        'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."
    }
  else:
    # if no context...
    systemPrmpt = {
        'role': 'system',
        'content': "You are humor classification model that takes a given line and determines whether it was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."
    }

  # append systemPrmpt
  messages.append(systemPrmpt)

  systemPrmpt2 = {
      'role': 'system',
      'content': ""
  }
  userPrmpt = {
      'role': 'user',
      'content': ""
  }
  userPrmpt2 = {
      'role': 'user',
      'content': ""
  }
  asstPrmpt = {
      'role': 'assistant',
      'content': ""
  }

  # append userPrmpt
  # messages.append(userPrmpt)

  if len(context) > 0:
    # asstPrmpt['content'] += "Can you provide some context to help me with this decision?"
    # append asstPrmpt
    # messages.append(asstPrmpt)

    userPrmpt['content'] += "Use the following as context:" + separator*2

    # if entry['Scene'] != current_scene:
    #   current_scene = entry["Scene"]
    # userPrmpt['content'] += "The scene changes to: "
    # else:

    entry = context[0]
    current_scene = entry["Scene"]
    userPrmpt['content'] += "The Scene is: "
    userPrmpt['content'] += current_scene + separator

    for entry in context:
      if current_scene != entry['Scene']:
        current_scene = entry["Scene"]
        userPrmpt['content'] += "The scene changes to: " + current_scene + separator

      recipients = entry['Recipients']
      recipients_str = ""
      if len(recipients) == 0:
        recipients_str += "to themselves"
      elif len(recipients) == 1:
        recipients_str += recipients[0]
      else:
        recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

      userPrmpt['content'] += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
      userPrmpt['content'] += separator

    # append userPrmpt2

  entry = target
  if len(context) > 0:
    # with context
    userPrmpt['content'] += separator * 2 + "Based on the context provided, is the following line from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:"

    current_scene = context[-1]['Scene']
    userPrmpt['content'] += separator*2


    if entry['Scene'] != current_scene:
      current_scene = entry["Scene"]
      userPrmpt['content'] += "The scene changes to: "
    else:
      userPrmpt['content'] += "The Scene is still: "

    userPrmpt['content'] += current_scene + separator

    recipients_str = ""
    recipients = entry['Recipients']
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    userPrmpt['content'] += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."

  else:
    # no context
    userPrmpt['content'] += "Is the following line humorous or not humorous? Only respond '1' for humorous or '0' for not humorous: "
    userPrmpt['content'] += f"{quotation_token}{entry['Dialogue']}{quotation_token}."

  messages.append(userPrmpt)
  return messages

### Sample Raw and Cleaned Formats:

In [6]:
filePath = "/content/info4940-sitcom/cleaned-data/S1/The Big Bang_S0101.json"
nSentenceContext = 5

with open(filePath, 'r') as file:
  data = json.load(file)
  data = [info for (_, info) in data.items()]

print("NCONTEXT: ", nSentenceContext)

print(f"Example raw entry:")
display(data[nSentenceContext])

print(f"\n\nExample formatted entry ({nSentenceContext}context):\n")
input_ = transformInput(data[:nSentenceContext], data[nSentenceContext])

for i in input_:
  print(f"Role: {i['role']} | Content:\n{i['content']}\n")

print(f'Actual: {("isHumor" in data[3])}')

NCONTEXT:  5
Example raw entry:


{'EpisodeID': 'The Big Bang_S0101',
 'Scene': 'A corridor at a sperm bank.',
 'Recipients': ['Sheldon', 'Receptionist'],
 'Speaker': 'Leonard',
 'Dialogue': '14 down is... Move your finger...',
 'Dialogue Start Time': '00:00:35:120000',
 'Dialogue End Time': '00:00:37:190000',
 'isHumor': True,
 'humorDuration': '0:00:01'}



Example formatted entry (5context):

Role: system | Content:
You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0.

Role: user | Content:
Use the following as context:

The Scene is: A corridor at a sperm bank.
Sheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it's unobserved, it will. However, if it's observed after it's left the plane but before it hits its target, it won't have gone through both slits." to Leonard.
Leonard says "Agreed. What's your point?" to Sheldon.
Sheldon says "There's no point, I just think it's a good idea for a T-shirt." to Leonard.
Leonard says "Excuse me." to Sheldon and Receptionist.
Leonard says "One across is Aegean. Eight down is Nabokov. 26 across is MCM." to She

# Creating sample data

In [7]:
filePaths = []
for i in range(5):
  fPath = '/content/info4940-sitcom/cleaned-data/S' + str(i+1)
  filePaths += list_files(fPath)

# getting random number for random state for consistency?
rand_seed1 = 64 # random.randint(1,100)
rand_seed2 = 94 # random.randint(1,100)
rand_seed3 = 69 # random.randint(1,100)

# print([rand_seed1,rand_seed2,rand_seed3])

# sampling 10 episodes
nSamples = 10
test_size = (len(filePaths) - nSamples) / len(filePaths)
samplePaths, _ = train_test_split(filePaths, test_size=test_size, random_state=rand_seed1)

# cleaning/transforming inputs
nContext = 5
nTests = 100

humorousData, nonHumorousData = fPathList_TO_DialogueWithContext(samplePaths, nContext)
sample_inputs, sample_labels = sampleEven(humorousData=humorousData, nonHumorousData=nonHumorousData, sample_size_per_class=(nTests/2), rand_seed=rand_seed2)

# shuffling, not necessary but helpful for watching in-progress results
rand_order = [i for i in range(len(sample_inputs))]
random.seed(rand_seed3)
random.shuffle(rand_order)

inputs = [sample_inputs[i] for i in rand_order]
labels = [sample_labels[i] for i in rand_order]

Sample input

In [11]:
responses = []
for i in range(len(inputs)):
  print(inputs[i][1]["content"])
  response = ""
  while response not in ["1", "0"]:
    response = input("Answer: ")
  responses.append(response)

Use the following as context:

The Scene is: The cafeteria.
Sheldon says "Professor Jones told the symposium he had a new method for calculating the mass of a muon. Five times the limit of (laughs) 'E' to the upsilon as..." to Howard, Leonard and Raj.
Sheldon says "Okay. No, no. I'll start over. Professor..." to Howard, Leonard and Raj.
Howard says "I haven't seen him laugh that hard since the day Leonard made that multiplication error." to Sheldon, Leonard and Raj.
Sheldon says "Oh, Oh, Lord, that multiplication error! He thought he carried the one. But he didn't." to Howard, Leonard and Raj.
Leonard says "It's not funny. That mistake got published." to Sheldon, Howard and Raj.


Based on the context provided, is the following line from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:

The Scene is still: The cafeteria.
Sheldon says "Stop! I'm going to wet myself!" to Howard, Leonard and Raj.
Answer: d
Answer: 4
Answer: 

KeyboardInterrupt: Interrupted by user

In [12]:
responses

['1', '0']